In [3]:
import numpy as np
import random

# Lista 7 (9 pkt.)

## Zadanie 1 (2 pkt.)

Prześledź działanie protokołu Diffiego-Helmana i uzupełnij poszczególne kroki.

Alicja i Bob ustalają publicznie grupę $Z_p$ w której będą działać oraz element z tej grupy $g$.

In [4]:
g=5
p=23

Alicja wybiera losowy element $a$ grupy $Z_p$, taki, że $1<a<p$:

In [32]:
a = random.randint(2, p-1)
print(a)

19


Następnie oblicza $A=g^a$ w grupie $Z_p$, czyli $A=g^a\mod\ p$:

In [34]:
A = pow(g, a, p)
print(A)

7


To samo robi Bob, wybiera losowy element grupy $b$ i oblicza $B=g^b$:

In [35]:
b=random.randint(2, p-1)
B=pow(g, b, p)

Elementy $a$ i $b$ są znane tylko, odpowiednio, Alicji i Bobowi, natomiast $A$ oraz $B$ są wymieniane publicznym kanałem. Bob otrzymuje $A$ i wykonuje operację $k_B=A^b$ (czyli $A^b\mod\ p$), z kolei Alicja otrzymuje $B$ i wykonuje operację $k_A=B^a$ (czyli $B^a\mod\ p$):

In [36]:
k_B=pow(A, b, p)
k_A=pow(B, a, p)

Zawuażmy, że $k_A=k_B$, wynika to z tego, że $k_A=B^a=(g^b)^a=g^{ab}$ oraz $k_B=A^b=(g^a)^b=g^{ba}$ a oczywiście $ab=ba$. Zatem Alicja i Bob dysponują tą samą liczbą, którą mogą użyć jako klucz do szyfrowania. Publicznie znane są wartości $g$, $p$, $g^a$ oraz $g^b$, jednak odzyskanie z nich $a$ i $b$ jest bardzo trudne przy odpwiednim doborze grupy $Z_p$.

## Zadanie 2 (2 pkt.)

Zaimplemetuj szybkie potęgowanie modulo. Napisz funkcję **pow\_mod(x,n,m)**, która oblicza $x^n\mod m$.

1. Niech $p=1$.
2. Iteruj po bitach reprezentujących $n$.
3. Przy każdej iteracji zastąp $p$ kwadratem, $\quad p\rightarrow p^2\mod m$.
4. W iteracjach, w ktorych bit jest jednyką, domnóż $x$ do $p$, $\quad p\rightarrow px\mod m$.
5. Na końcu $p$ będzie wynikiem, $p=x^n\mod m$

In [37]:
def pow_mod(x, n, m):
    p = 1
    while n > 0:
        if n % 2 == 1:
            p = (p * x) % m
        x = (x * x) % m
        n = n // 2
    return p

In [38]:
print(pow_mod(7,3,2)==1)
print(pow_mod(2,1024,7)==2)
print(pow_mod(3,10**100,7)==4)
print(pow_mod(3**99,10**100,7)==1)

True
True
True
True


## Zadanie 3 (1 pkt)

Napisz funkcję znajdującą dla danej liczby $d$ i $n$ taką liczbe $e$, że:
$$d\cdot e=1\mod n$$
czyli inaczej mówiąc odwotność $d$ w ciele $Z_n^*$. Użyj Rozszerzonego Algorytmu Euklidesa.

In [39]:
def inv(p, n):
    t0, t1 = 0, 1
    r0, r1 = n, p
    while r1 != 0:
        q = r0 // r1
        t0, t1 = t1, t0 - q * t1
        r0, r1 = r1, r0 - q * r1
    if r0 != 1:
        return None  # odwrotność nie istnieje
    return t0 % n

In [40]:
print(inv(5,7)==3)
print(inv(3,2)==1)
print(inv(5,7)==3)
print(inv(3,11)==4)

True
True
True
True


## Zadanie 4 (1 pkt.)

Zaimplementuj test Fermata, który dla danej liczby $p$ sprawdza za pomocą $k$ rund czy jest pierwsza i zwraca **True** lub **False**.

Male twierdzenie Fermata mówi, że jeżeli $p$ jest liczbą pierwszą i $a$ nie jest podzielne przez $p$ to $a^{p-1}$ jest równe $1$ modulo $p$:

$$p\in\mathbb{P}\ \wedge\ p\nmid a\ \Longrightarrow\ a^{p-1}=1\mod p$$

zatem biorąc zaprzeczenie powyższej implikacji mamy:

$$a^{p-1}\neq1\mod p\ \Longrightarrow\ p\notin\mathbb{P}\ \vee\ p\mid a$$

Tzn. jeżeli weźmiemy dowolne $a$ mniejsze od $p$ (w ten sposób wykluczamy $p\mid a$) i równość $a^{p-1}=1\mod p$ nie zajdzie to wiemy, że $p$ jest na pewno liczbą pierwszą, natomiast jezeli $a^{p-1}=1\mod p$ zachodzi wtedy jest duże prawdopodobieństwo, że $p$ jest liczbą pierwszą aczkolwiek nie jest to pewne.

Należy zatem wybrać liczbę $a\in[2,p-2]$, sprawdzić czy zachodzi $a^{p-1}=1\mod p$, jeżeli nie zachodzi to $p$ jest złożone a jesli zachodzi należy wziąć inne $a$ i ponownie sprawdzić, parametr $k$ określa ile razy $a$ bierzemy. Jeżeli po $k$ powtórzeniach za każdym razem równość zachodzi, możemy przyjąć, że $p$ jest pierwsze.

In [41]:
def Fermat_test(p, k):
    if p == 2:
        return True
    if p % 2 == 0:
        return False

    for _ in range(k):
        a = random.randint(2, p-2)
        if pow_mod(a, p-1, p) != 1:
            return False
    return True

In [42]:
print(Fermat_test(71,10)==True)
print(Fermat_test(41,10)==True)
print(Fermat_test(62,10)==False)
print(Fermat_test(84,10)==False)

True
True
True
True


Zaimplementuj funkcję **gen\_p(a,b)**, która zwraca losową liczbę pierwszą z przedziału $a$ i $b$, tzn. losuje liczby z tego przedziału i sprawdza czy są pierwsze testem Fermata tak długo aż znajdzie liczbę pierwszą.

In [43]:
def gen_p(a, b):
    while True:
        p = random.randint(a, b)
        if Fermat_test(p, 10):
            return p

## Zadanie 5 (2 pkt.)

Zaimplementuj generację kluczy w ramach algorytmu RSA, napisz funkcję **key\_gen(p,q)**, która dla podanych dużych liczb pierwszych $p$ i $q$ zwraca parę kluczy w postaci krotki $(n,e,d)$.

1. Obliczamy iloczyn $n=pq$
2. Następnie funkcję Eulera $\phi(n)=(q-1)(p-1)$
3. Klucz publiczny to para $(e,n)$ gdzie $e$ to liczba ze zbioru $\{1,2,..,\phi(n)-1\}$ taka, że $NWD(e,\phi(n))=1$, może być ona ustalona np. jako $e=2^{2^4}+1=65537$, w teście użyto właśnie tej liczby.
4. Klucz prywatny to para $(d,n)$, gdzie $d$ to liczba taka, że $de=1\ mod\ \phi(n)$, czyli $d$ jest odwrotnością $e$ w ciele $\mathbb{Z}_{\phi(n)}^*$.

In [44]:
def key_gen(p, q):
    n = p * q
    phi = (p-1) * (q-1)
    e = 65537
    d = inv(e, phi)
    return (n, e, d)

In [45]:
p=24130780476900131841553779066939443255102203937160657723394451174808141403858935238883126295228560935516885174421847238379397184900972008801015315248328437

In [46]:
q=26660613491521684005574100352062919789979599401844483402246984186988668019447679726081352452799126206997555710356464145743285983450292024894053538317854159

In [47]:
print(key_gen(p,q)==(
    643341411543391711051425916925550311012265711300705520200325675109446836493100912341600261266222036750541155307483726185012838542757173209246878527615686866322037404779287199511097525538499079836420404197380885254900993985365780000028685663116338197119892656788379026665075201747282243427197060237417498419483,
    65537,
    334692241429603741219438891581498052305769251366366399304669177607406348936208181733781847015759652456012644616150535488014598320266503205353805078033123914361616918116605669461614375732022492713408743728419283824726654095683796656269600488579712785553345684168299073769307373555258299179136288438930486131753))

True


## Zadanie 6 (1 pkt.)

Zaimplementuj funkcję **enc(x,e,n)**, która podaną liczbę $x$ (wiadomość) szyfruje za pomocą klucza publicznego $(e,n)$ oraz funkcję **dec(y,d,n)**, która podaną liczbę $y$ (szyfrogram) deszyfruje za pomocą klucza prywatnego $(d,n)$.

Szyfrowanie polega na wykonaniu potęgowania modulo:
$$y=x^e\ mod\ n$$
podobnie deszyfracja
$$x=y^d\ mod\ n$$

In [48]:
def enc(x, e, n):
    return pow_mod(x, e, n)

def dec(y, d, n):
    return pow_mod(y, d, n)

In [49]:
n=643341411543391711051425916925550311012265711300705520200325675109446836493100912341600261266222036750541155307483726185012838542757173209246878527615686866322037404779287199511097525538499079836420404197380885254900993985365780000028685663116338197119892656788379026665075201747282243427197060237417498419483
e=208350389615113762788111263490297665109355377830736643503856528470357220208290606069461253441671763980314762012190291145414733535673548961910772961435143582756267132618995046438684186252163655289035228721360753675271711075033036291412267917936062230585687839901652792581357105686274730618278123193067279319927
d=502029145905912565237092248595126620632487653124329465045136187249992350554283419049087834111437813928483679744364041267436534897197233494007405790027156754593648940515350675746678776751088177077690779849077150675864299782170211270887279535225267271652686426692746042361641530130191025648848746210219401813175

In [50]:
print(enc(17,e,n)==353230656531616665332116231509462661273082280099289165110086677972943261270362976411810450837847461343993316190457124231852161403281191913264230575248953060776390559207669288928802429515257729255854064666904850354451664771847425807841069296028397747015905377374208615536177338019721932982992946095124218548486)
print(dec(581228535329363957060482357417595500042117791982900743030228020443422357943293873902079555506233253640573184749108783275472243891683169424548126947970217999010556081853170166407244862004725833809785262442186726634369847615830487904940967188707443976155835347542897227831115870912021598488639913865347475436893,d,n)==27)
print(dec(enc(12,e,n),d,n)==12)

True
True
True
